Original

In [38]:
import numpy as np
from numba import jit
import random
import time

@jit(nopython=True)
def check_winner(board, x, y, player, rows, columns, inarow):
    directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
    for dx, dy in directions:
        # Check forward
        count = 1
        for step in range(1, inarow):
            nx, ny = x + step * dx, y + step * dy
            if 0 <= nx < rows and 0 <= ny < columns and board[nx, ny] == player:
                count += 1
            else:
                break

        if count >= inarow:
                return True

        # Check backward
        for step in range(1, inarow):
            nx, ny = x - step * dx, y - step * dy
            if 0 <= nx < rows and 0 <= ny < columns and board[nx, ny] == player:
                count += 1
            else:
                break

        # Only need to check if count is at least `inarow`
        if count >= inarow:
            return True

    return False

@jit(nopython=True)
def play_game(board, move, player, rows, columns, inarow):
    rows_available = np.where(board[:, move] == 0)[0]
    if rows_available.size == 0:
        return None  # Column full
    row = rows_available[-1]
    board[row, move] = player

    if check_winner(board, row, move, player, rows, columns, inarow):
        return player  # This move led directly to a win

    current_player = 3 - player  # Switch player
    while True:
        possible_moves = np.where(board[0, :] == 0)[0]
        if possible_moves.size == 0:
            return 0  # Draw game

        random_move = np.random.choice(possible_moves)  # Use np.random.choice
        rows_available = np.where(board[:, random_move] == 0)[0]
        random_row = rows_available[-1]
        board[random_row, random_move] = current_player

        if check_winner(board, random_row, random_move, current_player, rows, columns, inarow):
            return current_player  # This player wins

        current_player = 3 - current_player  # Switch player



def sim_games(board, player, rows, columns, inarow):
    valid_moves = [m for m in range(columns) if board[0, m] == 0]
    results = np.zeros(len(valid_moves), dtype=int)
    #print(results)
    num_sims = 500
    for idx, move in enumerate(valid_moves):
        sim_score = 0
        for _ in range(num_sims):
            board_copy = board.copy()
            result = play_game(board_copy, move, player, rows, columns, inarow)
            if result == player:
                sim_score += 1
            elif result == 0:
                continue
            elif result is not None:
                sim_score -= 1
        results[idx] = sim_score
    #print(results)
    if results.size > 0:
        best_idx = np.argmax(results)
        best_move = valid_moves[best_idx]
    else:
        print('used random')
        best_move = np.random.choice(valid_moves)  # Use np.random.choice
    return best_move

def agent(observation, configuration):

    board = np.array(observation['board']).reshape(configuration['rows'], configuration['columns'])
    player = observation['mark']
    import time
    t1 = time.time()
    best_move = sim_games(board, player, configuration['rows'], configuration['columns'], configuration['inarow'])

    #print(time.time() - t1)
    #print(best_move)
    #print('-'*26)
    return int(best_move)


Better

In [33]:
import numpy as np
from numba import jit
import time
import random
from dataclasses import dataclass
from typing import List, Optional

BUDGET = 10_000_000
N_SIM = 500  # Number of simulations per leaf node
MOVES_PER_GAME = 42

# Formula to calculate the total number of moves for exhaustive + Monte Carlo on leaf nodes
def calculate_total_moves_combined(b, d, n_sim=N_SIM, moves_per_game=MOVES_PER_GAME):
    # Handle edge cases
    if b == 0:  # No valid moves
        return 0
    elif b == 1:  # Only one valid move
        return d + 1 + n_sim * moves_per_game  # Linear path + Monte Carlo on leaf
    else:
        # Normal case: sum of geometric series for exhaustive moves + Monte Carlo simulations
        exhaustive_moves = (b**(d+1) - 1) / (b - 1)
        monte_carlo_moves = b**d * n_sim * moves_per_game
        return exhaustive_moves + monte_carlo_moves


# Function to dynamically calculate maximum depth within budget
def calculate_max_depth(b, budget=BUDGET):
    max_depth = 0
    while calculate_total_moves_combined(b, max_depth) <= budget:
        max_depth += 1
    return max_depth - 1  # Subtract 1 because we go one step too far in the loop

@jit(nopython=True)
def check_winner(board, x, y, player, rows, columns, inarow):
    directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
    for dx, dy in directions:
        count = 1
        for step in range(1, inarow):
            nx, ny = x + step * dx, y + step * dy
            if 0 <= nx < rows and 0 <= ny < columns and board[nx, ny] == player:
                count += 1
            else:
                break
        if count >= inarow:
            return True
        for step in range(1, inarow):
            nx, ny = x - step * dx, y - step * dy
            if 0 <= nx < rows and 0 <= ny < columns and board[nx, ny] == player:
                count += 1
            else:
                break
        if count >= inarow:
            return True
    return False

@jit(nopython=True)
def play_game(board, move, player, rows, columns, inarow):
    rows_available = np.where(board[:, move] == 0)[0]
    if rows_available.size == 0:
        return None
    row = rows_available[-1]
    board[row, move] = player

    if check_winner(board, row, move, player, rows, columns, inarow):
        return player

    current_player = 3 - player
    while True:
        possible_moves = np.where(board[0, :] == 0)[0]
        if possible_moves.size == 0:
            return 0
        random_move = np.random.choice(possible_moves)
        rows_available = np.where(board[:, random_move] == 0)[0]
        random_row = rows_available[-1]
        board[random_row, random_move] = current_player

        if check_winner(board, random_row, random_move, current_player, rows, columns, inarow):
            return current_player

        current_player = 3 - current_player


@dataclass
class GameNode:
    board: np.ndarray
    move: Optional[int]  # The move that led to this state (None for root)
    player: int  # Player who will make the next move
    value: float = 0.0
    children: List['GameNode'] = None

    def __post_init__(self):
        if self.children is None:
            self.children = []

def build_game_tree(board: np.ndarray, depth: int, player: int, rows: int, columns: int, inarow: int, move: Optional[int] = None) -> GameNode:
    """
    Recursively builds the game tree up to the specified depth.
    Returns the root node of the tree.
    """
    node = GameNode(board.copy(), move, player)

    # Base case: reached depth limit or game over
    if depth == 0:
        return node

    valid_moves = [m for m in range(columns) if board[0, m] == 0]

    # Game over check
    if not valid_moves:
        return node

    # Recursively build children
    for move in valid_moves:
        board_copy = board.copy()
        rows_available = np.where(board_copy[:, move] == 0)[0]
        row = rows_available[-1]
        board_copy[row, move] = player

        # Check if this move results in a win
        if check_winner(board_copy, row, move, player, rows, columns, inarow):
            child = GameNode(board_copy, move, 3 - player)
            child.value = float('inf') if player == 1 else float('-inf')
            node.children.append(child)
            continue

        # Recurse
        child = build_game_tree(board_copy, depth - 1, 3 - player, rows, columns, inarow, move)
        node.children.append(child)

    return node

def monte_carlo_leaf_evaluation(node: GameNode, rows: int, columns: int, inarow: int, num_sims: int = 100) -> float:
    """
    Performs Monte Carlo simulations from a leaf node position.
    Returns the evaluation score.
    """
    wins = 0
    for _ in range(num_sims):
        board_copy = node.board.copy()
        current_player = node.player

        # Play random moves until game ends
        while True:
            valid_moves = [m for m in range(columns) if board_copy[0, m] == 0]
            if not valid_moves:
                break

            move = random.choice(valid_moves)
            rows_available = np.where(board_copy[:, move] == 0)[0]
            row = rows_available[-1]
            board_copy[row, move] = current_player

            if check_winner(board_copy, row, move, current_player, rows, columns, inarow):
                if current_player == 1:
                    wins += 1
                else:
                    wins -= 1
                break

            current_player = 3 - current_player

    return wins / num_sims

def evaluate_tree(node: GameNode, rows: int, columns: int, inarow: int) -> float:
    """
    Recursively evaluates the game tree using minimax principles.
    Performs Monte Carlo simulation on leaf nodes and propagates values upward.
    """
    # Base case: leaf node
    if not node.children:
        # If value is already set (winning position), return it
        if abs(node.value) == float('inf'):
            return node.value
        # Otherwise, evaluate with Monte Carlo
        node.value = monte_carlo_leaf_evaluation(node, rows, columns, inarow)
        return node.value

    # Recursive case: evaluate children and propagate values
    if node.player == 1:  # Maximizing player
        node.value = max(evaluate_tree(child, rows, columns, inarow) for child in node.children)
    else:  # Minimizing player
        node.value = min(evaluate_tree(child, rows, columns, inarow) for child in node.children)

    return node.value

def agent2(observation, configuration):
    board = np.array(observation['board']).reshape(configuration['rows'], configuration['columns'])
    player = observation['mark']

    # Calculate branching factor and maximum depth
    b = len([m for m in range(configuration['columns']) if board[0, m] == 0])
    max_depth = calculate_max_depth(b)

    # Build the game tree
    root = build_game_tree(board, max_depth, player, configuration['rows'], configuration['columns'], configuration['inarow'])

    # Evaluate the tree
    evaluate_tree(root, configuration['rows'], configuration['columns'], configuration['inarow'])

    # Choose the best move based on children's values
    if player == 1:  # Maximizing player
        best_child = max(root.children, key=lambda x: x.value)
    else:  # Minimizing player
        best_child = min(root.children, key=lambda x: x.value)

    return int(best_child.move)

Speed Optimised

In [40]:
import numpy as np
from numba import njit
import random

BUDGET = 40_000_000
N_SIM = 500  # Number of simulations per leaf node
MOVES_PER_GAME = 42

# Formula to calculate the total number of moves for exhaustive + Monte Carlo on leaf nodes
def calculate_total_moves_combined(b, d, n_sim=N_SIM, moves_per_game=MOVES_PER_GAME):
    # Handle edge cases
    if b == 0:  # No valid moves
        return 0
    elif b == 1:  # Only one valid move
        return d + 1 + n_sim * moves_per_game  # Linear path + Monte Carlo on leaf
    else:
        # Normal case: sum of geometric series for exhaustive moves + Monte Carlo simulations
        exhaustive_moves = (b ** (d + 1) - 1) // (b - 1)
        monte_carlo_moves = b ** d * n_sim * moves_per_game
        return exhaustive_moves + monte_carlo_moves

# Function to dynamically calculate maximum depth within budget
def calculate_max_depth(b, budget=BUDGET):
    max_depth = 0
    while calculate_total_moves_combined(b, max_depth) <= budget:
        max_depth += 1
    return max_depth - 1  # Subtract 1 because we go one step too far in the loop

@njit
def check_winner(board, x, y, player, rows, columns, inarow):
    # Directions: vertical, horizontal, diagonal /, diagonal \
    directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
    for dx, dy in directions:
        count = 1
        for d in [1, -1]:
            step = 1
            while True:
                nx, ny = x + step * dx * d, y + step * dy * d
                if 0 <= nx < rows and 0 <= ny < columns and board[nx, ny] == player:
                    count += 1
                    step += 1
                    if count >= inarow:
                        return True
                else:
                    break
        # Reset count for next direction
    return False

@njit
def play_game(board, player, rows, columns, inarow):
    current_player = player
    while True:
        # Get list of valid moves
        valid_moves = []
        for m in range(columns):
            if board[0, m] == 0:
                valid_moves.append(m)
        if len(valid_moves) == 0:
            return 0  # Draw
        move = valid_moves[np.random.randint(0, len(valid_moves))]

        # Place the piece
        for r in range(rows - 1, -1, -1):
            if board[r, move] == 0:
                board[r, move] = current_player
                x, y = r, move
                break

        # Check for a win
        if check_winner(board, x, y, current_player, rows, columns, inarow):
            return current_player

        current_player = 3 - current_player  # Switch player

@njit
def monte_carlo_leaf_evaluation(board, player, rows, columns, inarow, num_sims):
    wins = 0
    for _ in range(num_sims):
        board_copy = board.copy()
        result = play_game(board_copy, player, rows, columns, inarow)
        if result == 1:
            wins += 1
        elif result == 2:
            wins -= 1
    return wins / num_sims

class GameNode:
    def __init__(self, board, move, player):
        self.board = board
        self.move = move
        self.player = player
        self.value = 0.0
        self.children = []

def build_game_tree(board, depth, player, rows, columns, inarow, move=None):
    node = GameNode(board.copy(), move, player)

    # Base case: reached depth limit or game over
    if depth == 0:
        return node

    valid_moves = [m for m in range(columns) if board[0, m] == 0]

    # Game over check
    if not valid_moves:
        return node

    # Recursively build children
    for move in valid_moves:
        board_copy = board.copy()
        for r in range(rows - 1, -1, -1):
            if board_copy[r, move] == 0:
                board_copy[r, move] = player
                x, y = r, move
                break

        # Check if this move results in a win
        if check_winner(board_copy, x, y, player, rows, columns, inarow):
            child = GameNode(board_copy, move, 3 - player)
            child.value = float('inf') if player == 1 else float('-inf')
            node.children.append(child)
            continue

        # Recurse
        child = build_game_tree(board_copy, depth - 1, 3 - player, rows, columns, inarow, move)
        node.children.append(child)

    return node

def evaluate_tree(node, rows, columns, inarow, num_sims):
    """
    Recursively evaluates the game tree using minimax principles.
    Performs Monte Carlo simulation on leaf nodes and propagates values upward.
    """
    # Base case: leaf node
    if not node.children:
        # If value is already set (winning position), return it
        if abs(node.value) == float('inf'):
            return node.value
        # Otherwise, evaluate with Monte Carlo
        node.value = monte_carlo_leaf_evaluation(node.board, node.player, rows, columns, inarow, num_sims)
        return node.value

    # Recursive case: evaluate children and propagate values
    child_values = []
    for child in node.children:
        child_value = evaluate_tree(child, rows, columns, inarow, num_sims)
        child_values.append(child_value)

    if node.player == 1:  # Maximizing player
        node.value = max(child_values)
    else:  # Minimizing player
        node.value = min(child_values)
    return node.value

def agent2(observation, configuration):
    board = np.array(observation['board']).reshape(configuration['rows'], configuration['columns'])
    player = observation['mark']
    rows = configuration['rows']
    columns = configuration['columns']
    inarow = configuration['inarow']

    # Calculate branching factor and maximum depth
    b = np.count_nonzero(board[0, :] == 0)
    max_depth = calculate_max_depth(b)
    # print(max_depth)
    # Build the game tree
    root = build_game_tree(board, max_depth, player, rows, columns, inarow)

    # Evaluate the tree
    num_sims = N_SIM  # You can adjust this based on your needs
    evaluate_tree(root, rows, columns, inarow, num_sims)

    # Choose the best move based on children's values
    best_value = float('-inf') if player == 1 else float('inf')
    best_move = None
    for child in root.children:
        if (player == 1 and child.value > best_value) or (player == 2 and child.value < best_value):
            best_value = child.value
            best_move = child.move
    # print('Player: ', player)
    # print(len(root.children))
    # print(best_move)
    # print(best_value)
    # for child in root.children:
    #   print(child.value)
    # print('-'*15)
    # If best_move is still None, select the first available move
    if best_move is None:
        valid_moves = [c for c in range(columns) if board[0, c] == 0]
        best_move = valid_moves[0] if valid_moves else 0  # Default to column 0 if no valid moves


    return int(best_move)


In [41]:
import numpy as np
import random
from tqdm import tqdm

# Assume your previous functions: check_winner, play_game, sim_games


def play_game_two_bots(board, bot1_agent, bot2_agent, player, rows, columns, inarow):
    """
    Simulates a game between two bots using their agent functions.
    """
    current_player = player
    while True:
        possible_moves = np.where(board[0, :] == 0)[0]
        if possible_moves.size == 0:
            return 0  # Draw game

        observation = {'board': board.flatten(), 'mark': current_player}
        configuration = {'rows': rows, 'columns': columns, 'inarow': inarow}

        if current_player == 1:
            move = bot1_agent(observation, configuration)
        else:
            move = bot2_agent(observation, configuration)

        rows_available = np.where(board[:, move] == 0)[0]
        if rows_available.size == 0:
            continue  # Invalid move, try again
        row = rows_available[-1]
        board[row, move] = current_player

        if check_winner(board, row, move, current_player, rows, columns, inarow):
            return current_player  # Return the winner

        current_player = 3 - current_player  # Switch player



def simulate_multiple_games(bot1_agent, bot2_agent, num_games, rows, columns, inarow):
    """
    Simulate a number of games between two agents and collect statistics.
    """
    bot1_wins = 0
    bot2_wins = 0
    draws = 0

    # Wrap the loop with tqdm to display the progress bar
    for game in tqdm(range(num_games), desc="Simulating games"):
        board = np.zeros((rows, columns), dtype=int)
        starting_player = random.choice([1, 2])  # Randomize starting player

        result = play_game_two_bots(board, bot1_agent, bot2_agent, starting_player, rows, columns, inarow)

        if result == 1:
            bot1_wins += 1
        elif result == 2:
            bot2_wins += 1
        else:
            draws += 1

    # Print final statistics
    print(f"Total games: {num_games}")
    print(f"Bot1 wins: {bot1_wins} ({(bot1_wins / num_games) * 100:.2f}%)")
    print(f"Bot2 wins: {bot2_wins} ({(bot2_wins / num_games) * 100:.2f}%)")
    print(f"Draws: {draws} ({(draws / num_games) * 100:.2f}%)")


# Define both agents using the same agent function or different ones if desired
def agent_bot1(observation, configuration):
    # This is where you define the first bot's strategy (using the agent function)
    return agent(observation, configuration)  # Calls the previously defined `agent` function

def agent_bot2(observation, configuration):
    # This is where you define the second bot's strategy (could be different from agent_bot1)
    return agent2(observation, configuration)  # Calls the same agent function, can be customized

# Example usage:
simulate_multiple_games(agent_bot1, agent_bot2, num_games=100, rows=6, columns=7, inarow=4)


Simulating games:   0%|          | 0/100 [00:00<?, ?it/s]


TypeError: too many arguments: expected 5, got 6